In [17]:
from dash import Dash, dash_table, dcc, html
from dash.dependencies import Input, Output
import pandas as pd
from flask import Flask
from Librerias_SAP import SAP_GUI, Funtions
import pandas as pd
import re
from collections import OrderedDict
from dash import dash_table as dt
from dash import dcc
from dash import html
from dash.dependencies import Input
from dash.dependencies import Output
import numpy as np
import dash_bootstrap_components as dbc
from datetime import datetime, timedelta

In [18]:
"""----------------------------Inciar session----------------------------------------------------"""
# Insert User name and password

Keys=pd.read_excel("C:\\Users\\practicante.picking\\OneDrive - Prebel S.A\\Escritorio\\bot_picking\\Archivos_CSV\\Keys.xlsx")

user=Keys["User"][0]
password=Keys["Password"][0]
# Initialize session
session=SAP_GUI.SessionSAP(user,password)

In [19]:
Defaul_Column_Pedidos_dia=['Documento', 'GTr', 'Denomin.', 'ClVt', 'Denominación', 'Solic.',
       'Creado el', 'Fecha doc.', 'Pedido', 'Func.', 'Responsab', 'Creado',
       'OrgVt', 'CDis', 'Se', 'OfVta', 'GVen', 'Mon.', 'Valor neto']

In [20]:
# PEDIDOSEXITO


def Search_VA05N(Transsaccion,session,provision,variant):
    session.StartTransaction(Transsaccion)
    session.findById("wnd[0]/tbar[1]/btn[17]").press()
    session.findById("wnd[1]/usr/txtV-LOW").text = provision
    session.findById("wnd[1]/usr/txtENAME-LOW").text = variant
    session.findById("wnd[1]/usr/txtV-LOW").caretPosition = 8
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    return(session)

In [21]:
Transsaccion='va05n'
provision='PEDIDOSEXITO'
variant='JESPINOSAP'

Search_VA05N(Transsaccion,session,provision,variant)

<COMObject <unknown>>

In [22]:
Name_VA05N="Pedidos_diarios"
#
#
Ruta_Exito_ZSD010_ZVMI="C:\\Users\\practicante.picking\\OneDrive - Prebel S.A\\Escritorio\\Bot_Disnal_Pendientes_2\\Archivos_CSV\\" 
# Ruta_Exito_ZSD010_ZVMI="Archivos_CSV\\"   #Ruta prueva
#
#
#
SAP_GUI.Export_TXT2(Name_VA05N,session,Ruta_Exito_ZSD010_ZVMI)

In [23]:
Pedidos_VN05N=pd.read_csv(Ruta_Exito_ZSD010_ZVMI+Name_VA05N+".txt",delimiter="\t",skiprows=1)
Pedidos_VN05N=Funtions.Clean_Columns(Pedidos_VN05N)
Pedidos_VN05N=Funtions.default_column(Defaul_Column_Pedidos_dia,Pedidos_VN05N)
#Elimino pedidos con valores nulos
Pedidos_VN05N=Pedidos_VN05N[Pedidos_VN05N['Pedido'].notnull()]

In [24]:
Pedidos_VN05N['Pedido']

0       0020666499
1       0020666501
2       0020666552
3       0020666499
4       0020666499
           ...    
4757    0138039487
4758    4816007492
4759    0435090782
4760    0085257047
4761    0384016492
Name: Pedido, Length: 4748, dtype: object

In [25]:
Agenda=["85","20","146","149","50","138","45"]

Pedidos_VN05N['Pedido']=Funtions.Estandarizo_Pedidos(Pedidos_VN05N['Pedido'])
Pedidos_VN05N['Pedido']=Funtions.complete_pedidos(Pedidos_VN05N['Pedido'],Agenda)


In [26]:
Pedidos_VN05N['Pedido'].value_counts().keys()[50:100]

Index(['0146342321', '0085775855', '0085775846', '0020670958', '0085780948',
       '0085775841', '0085775742', '0146339264', '0085780973', '0020670955',
       '0085775057', '0146339262', '0020670868', '0050448374', '0020678229',
       '0085775168', '0020674335', '0085778709', '0138093983', '0050449379',
       '0138090770', '0020670959', '0050449014', '0085775741', '0020679423',
       '0020666563', '0020678265', '0085774056', '0138094035', '0085775811',
       '0085775848', '0020670964', '0085782659', '0138094037', '0020670965',
       '0050448944', '0085779371', '0138090829', '0050452526', '0146340070',
       '0020666552', '0085775805', '0146334047', '0020670918', '0020670919',
       '0085775812', '0085774057', '0146334046', '0020678262', '0146334053'],
      dtype='object', name='Pedido')

# Filtro para tener en cuenta las mayas de trabajo

In [27]:
# Maya de trabajo
# Dia lunes
Agenda_Exito_Lunes=["0020"]
Agenda_Cencosub_Lunes=["197-"]
# Dia Martes
Agenda_Exito_Martes=["0085"]
Agenda_Cencosub_Martes=["60-"]
# Dia Miercoles
Agenda_Exito_Miercoles=["0050","0146"]
Agenda_Cencosub_Miercoles=["0085"]
# Dia Jueves
Agenda_Exito_Jueves=["0085","138"]
Agenda_Cencosub_Jueves=["0085"]
# Dia Viernes
Agenda_Exito_Viernes=["0085"]
Agenda_Cencosub_Viernes=["0085"]


In [28]:
now=datetime.now()-timedelta(days=3)   #Dia actual
day=now.strftime("%A") #Obtengo dia
if day=="Monday":
    Maya_Exito=Funtions.Search_Agenda_Exito(Pedidos_VN05N['Pedido'],Agenda_Exito_Lunes)
    Maya_Cencosub=Funtions.Search_agenda_Cencosub(Pedidos_VN05N['Pedido'],Agenda_Cencosub_Lunes)
    filtro_Maya_Exito=Pedidos_VN05N[Pedidos_VN05N['Pedido'].isin(Maya_Exito)]
    filtro_Maya_Cencosub=Pedidos_VN05N[Pedidos_VN05N['Pedido'].isin(Maya_Cencosub)]
    print(filtro_Maya_Exito['Pedido'],filtro_Maya_Cencosub['Pedido'])
elif day=="Thursday":
    Maya_Exito=Funtions.Search_Agenda_Exito(Pedidos_VN05N['Pedido'],Agenda_Exito_Martes)
    Maya_Cencosub=Funtions.Search_agenda_Cencosub(Pedidos_VN05N['Pedido'],Agenda_Cencosub_Martes)
    filtro_Maya_Exito=Pedidos_VN05N[Pedidos_VN05N['Pedido'].isin(Maya_Exito)]
    filtro_Maya_Cencosub=Pedidos_VN05N[Pedidos_VN05N['Pedido'].isin(Maya_Cencosub)]
    print(filtro_Maya_Exito['Pedido'],filtro_Maya_Cencosub['Pedido'])
elif day=="Wednesday":
    Maya_Exito=Funtions.Search_Agenda_Exito(Pedidos_VN05N['Pedido'],Agenda_Exito_Miercoles)
    Maya_Cencosub=Funtions.Search_agenda_Cencosub(Pedidos_VN05N['Pedido'],Agenda_Cencosub_Miercoles)
    filtro_Maya_Exito=Pedidos_VN05N[Pedidos_VN05N['Pedido'].isin(Maya_Exito)]
    filtro_Maya_Cencosub=Pedidos_VN05N[Pedidos_VN05N['Pedido'].isin(Maya_Cencosub)]
    print(filtro_Maya_Exito['Pedido'],filtro_Maya_Cencosub['Pedido'])
elif day=="Thursday":
    Maya_Exito=Funtions.Search_Agenda_Exito(Pedidos_VN05N['Pedido'],Agenda_Exito_Jueves)
    Maya_Cencosub=Funtions.Search_agenda_Cencosub(Pedidos_VN05N['Pedido'],Agenda_Cencosub_Jueves)
    filtro_Maya_Exito=Pedidos_VN05N[Pedidos_VN05N['Pedido'].isin(Maya_Cencosub)]
    filtro_Maya_Cencosub=Pedidos_VN05N[Pedidos_VN05N['Pedido'].isin(Maya_Cencosub)]
    print(filtro_Maya_Exito['Pedido'],filtro_Maya_Cencosub['Pedido'])
else:
    Maya_Exito=Funtions.Search_Agenda_Exito(Pedidos_VN05N['Pedido'],Agenda_Exito_Viernes)
    Maya_Cencosub=Funtions.Search_agenda_Cencosub(Pedidos_VN05N['Pedido'],Agenda_Cencosub_Viernes)
    filtro_Maya_Exito=Pedidos_VN05N[Pedidos_VN05N['Pedido'].isin(Maya_Exito)]
    filtro_Maya_Cencosub=Pedidos_VN05N[Pedidos_VN05N['Pedido'].isin(Maya_Cencosub)]
    print(filtro_Maya_Exito['Pedido'],filtro_Maya_Cencosub['Pedido'])
    

188     0085774002
191     0085774002
192     0085774002
196     0085774002
198     0085774003
           ...    
4708    0085782604
4709    0085782604
4710    0085782605
4726    0085782771
4760    0085257047
Name: Pedido, Length: 1948, dtype: object Series([], Name: Pedido, dtype: object)


In [29]:
Lunes=["0085"]


buscar=Funtions.Search_Agenda_Exito(Pedidos_VN05N['Pedido'],Lunes)
filtro_Maya_Exito=Pedidos_VN05N[Pedidos_VN05N['Pedido'].isin(buscar)]

In [30]:
# filtro_cencosub['Pedido'].value_counts()

In [31]:
# filtro_cencosub['Pedido'].drop_duplicates().value_counts()

In [32]:
session.findById("wnd[0]/usr/btn%_SO_DOC_E_%_APP_%-VALU_PUSH").press()

com_error: (-2147352567, 'Ocurrió una excepción.', (619, 'SAP Frontend Server', 'The control could not be found by id.', 'C:\\Program Files (x86)\\SAP\\FrontEnd\\SapGui\\sapfront.HLP', 393215, 0), None)

In [ ]:
def Search_VN05N(Transsaccion,Series,session):      #(column Dataframe)
    """
    Transsaccion: Transsacion a buscar
    Series: Columna del dataframe que quiero copiar
    session: session del usuario
    provision: disposicion de interes
    """
    session.StartTransaction(Transsaccion)
    Series=Series.to_clipboard(index=False, header=False)
    session.findById("wnd[0]/usr/btn%_SO_BSTKD_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    session.findById("wnd[0]/tbar[1]/btn[8]").press()


In [ ]:
Search_VN05N('zsd79',filtro_cencosub['Pedido'].drop_duplicates(),session)

In [ ]:

session.StartTransaction('zsd79')
Series=filtro_cencosub['Pedido'].to_clipboard(index=False, header=False)
session.findById("wnd[0]/usr/btn%_SO_BSTKD_%_APP_%-VALU_PUSH").press()
session.findById("wnd[1]/tbar[0]/btn[24]").press()
session.findById("wnd[1]/tbar[0]/btn[8]").press()
session.findById("wnd[0]/tbar[1]/btn[8]").press()
table=session.findById("wnd[0]/usr/cntlGRID1/shellcont/shell")

# Nota no tiene entrega generada si Pedido Concluido Rojo y entrega conlcuida Verda

In [ ]:
table.SelectColumn("LFGSK")
table.SelectColumn("WBSTK")

In [ ]:
#Columna Pedido concluido 1
#table.GetCellValue(2,"LFGSK")
table.SelectColumn("LFGSK")
table.SelectColumn("BUCHK")
#Columna entregas concluidas
#Columna entregas
#table.SelectColumn("ZAD")
#Columna entregas concluidas
table.SelectColumn("WBSTK")

In [ ]:
table.SelectColumn("LFGSK")

In [ ]:
session.findById("wnd[0]/mbar/menu[1]/menu[3]").select() 
session.findById("wnd[1]/usr/ssub%_SUBSCREEN_FREESEL:SAPLSSEL:1105/ctxt%%DYN001-LOW").text = "@0A@"
session.findById("wnd[1]/usr/ssub%_SUBSCREEN_FREESEL:SAPLSSEL:1105/ctxt%%DYN002-LOW").text = "@08@" 
session.findById("wnd[1]/tbar[0]/btn[0]").press()

In [ ]:
#Columna Pedido concluido 1
table.GetCellValue(2,"LFGSK")

#Columna entregas concluidas
table.SelectColumn("WBSTK")

In [ ]:
table.GetCellValue(2,"LFGSK")

'@0A@'

In [ ]:
table.GetCellValue(2,"BSTNK")   #Nº Pedido cliente
table.GetCellValue(2,"AUART")   #Clase Orden
table.GetCellValue(2,"AUDAT")   #PrimFecEnt
table.GetCellValue(2,"VDATU")   #ÚltEntrega

'06.07.2023'

In [ ]:
Row=table.RowCount
for i in range(0,Row):
    Pedido_cliente,Clase_Orden,PrimFecEnt,ÚltEntrega=table.GetCellValue(i,"BSTNK"),table.GetCellValue(2,"AUART"),table.GetCellValue(2,"AUDAT"),table.GetCellValue(2,"VDATU") 
    print(Pedido_cliente,Clase_Orden,PrimFecEnt,ÚltEntrega)

0085770424 ZVMI 29.06.2023 06.07.2023
0085770425 ZVMI 29.06.2023 06.07.2023
0085770426 ZVMI 29.06.2023 06.07.2023
0085770427 ZVMI 29.06.2023 06.07.2023
0085770428 ZVMI 29.06.2023 06.07.2023
0085770429 ZVMI 29.06.2023 06.07.2023
0085770430 ZVMI 29.06.2023 06.07.2023
0085770431 ZVMI 29.06.2023 06.07.2023
0085770432 ZVMI 29.06.2023 06.07.2023
0085770433 ZVMI 29.06.2023 06.07.2023
0085770589 ZVMI 29.06.2023 06.07.2023
0085770620 ZVMI 29.06.2023 06.07.2023


In [ ]:
"@0A@       Filtro Rojo"      "WBSTK"   "BUCHK"
#Columna entregas concluidas
table.SelectColumn("BUCHK")


In [ ]:
#Columna entregas
table.GetCellValue(2,"ZAD")
"LFGSK"

'LFGSK'

In [ ]:
from time import sleep

In [ ]:
sleep(1)
session.findById("wnd[0]/mbar/menu[0]/menu[3]/menu[2]").select()
sleep(3)
session.findById("wnd[1]/usr/subSUBSCREEN_STEPLOOP:SAPLSPO5:0150/sub:SAPLSPO5:0150/radSPOPLI-SELFLAG[1,0]").select()
sleep(3)

In [ ]:
def Export_TXT(Name,session,Ruta=None):
    try:
        session.findById("wnd[0]/mbar/menu[0]/menu[3]/menu[2]").select()
    except:
        pass
    finally:
        session.findById("wnd[1]/usr/subSUBSCREEN_STEPLOOP:SAPLSPO5:0150/sub:SAPLSPO5:0150/radSPOPLI-SELFLAG[1,0]").select()
        session.findById("wnd[1]/tbar[0]/btn[0]").press()
        if Ruta==None:
            session.findById("wnd[1]/usr/ctxtDY_PATH").text = "C:\\Users\\practicante.picking\\OneDrive - Prebel S.A\\Escritorio\\bot_picking\\Archivos_CSV\\"
        else:
            session.findById("wnd[1]/usr/ctxtDY_PATH").text = Ruta
        session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = str(Name) + ".txt"
        session.findById("wnd[1]/usr/ctxtDY_FILE_ENCODING").text = "4310"
        session.findById("wnd[1]/tbar[0]/btn[11]").press()

In [ ]:
Name_VA05N="Avisos_despacho"
Ruta_Exito_ZSD010_ZVMI="C:\\Users\\practicante.picking\\OneDrive - Prebel S.A\\Escritorio\\bot_picking\\Archivos_CSV\\"
Export_TXT(Name_VA05N,session,Ruta_Exito_ZSD010_ZVMI)

In [ ]:
pd.read_csv(Ruta_Exito_ZSD010_ZVMI+Name_VA05N+".txt",delimiter='\t',skiprows=1)[['Fecha doc.', 'Unnamed: 8', 'FePrefEnt.',
       'Unnamed: 10', 'Pedido Concluido', 'Entregas Contab.',
       'Fact. Gen-Cont.', 'Aviso D.', 'Fec. AD', 'Unnamed: 16', 'Reg. AD',
       'F.Re. AD', 'Unnamed: 19', 'Borra. AD', 'F. Borr AD']]

,Fecha doc.,Unnamed: 8,FePrefEnt.,Unnamed: 10,Pedido Concluido,Entregas Contab.,Fact. Gen-Cont.,Aviso D.,Fec. AD,Unnamed: 16,Reg. AD,F.Re. AD,Unnamed: 19,Borra. AD,F. Borr AD
0,NaN,14.06.2023,NaN,17.06.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,14.06.2023,NaN,17.06.2023,NaN,NaN,NaN,NaN,NaN,16.06.2023,NaN,NaN,NaN,NaN,NaN
2,NaN,14.06.2023,NaN,17.06.2023,NaN,NaN,NaN,NaN,NaN,16.06.2023,NaN,NaN,NaN,NaN,NaN
3,NaN,14.06.2023,NaN,17.06.2023,NaN,NaN,NaN,NaN,NaN,16.06.2023,NaN,NaN,NaN,NaN,NaN
4,NaN,14.06.2023,NaN,17.06.2023,NaN,NaN,NaN,NaN,NaN,16.06.2023,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,NaN,28.06.2023,NaN,06.07.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,NaN,28.06.2023,NaN,01.07.2023,NaN,NaN,NaN,NaN,NaN,20.03.2019,NaN,NaN,NaN,NaN,NaN
152,NaN,28.06.2023,NaN,01.07.2023,NaN,NaN,NaN,NaN,NaN,20.03.2019,NaN,NaN,NaN,NaN,NaN
153,NaN,28.06.2023,NaN,08.07.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
prueba = pd.read_html("C:\\Users\\practicante.picking\\OneDrive - Prebel S.A\\Escritorio\\bot_picking\\Archivos_CSV\\Prueba.htm")